In [1]:
import requests
from bs4 import BeautifulSoup
import pandas
import json
payload = {}
headers = {
  'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
  'accept-language': 'zh-CN,zh;q=0.9',
  'cookie': 'bid=Gf8N7P12IX4; dbcl2="284492976:KLb0R0EShqA"; ck=13t7; _pk_ref.100001.8cb4=%5B%22%22%2C%22%22%2C1730562677%2C%22https%3A%2F%2Faccounts.douban.com%2F%22%5D; _pk_id.100001.8cb4=21d1d1676cf974fb.1730562677.; _pk_ses.100001.8cb4=1; frodotk_db="1baa55bf79c48a7952ff013e6d4d9f6b"; push_noty_num=0; push_doumail_num=0; ap_v=0,6.0; __utma=30149280.765760.1730562679.1730562679.1730562679.1; __utmc=30149280; __utmz=30149280.1730562679.1.1.utmcsr=accounts.douban.com|utmccn=(referral)|utmcmd=referral|utmcct=/; __utmt=1; __utmv=30149280.28449; __yadk_uid=bfc7tcSoGAJaCTBf6xXlS1wh1vmjTXkB; __utmb=30149280.20.5.1730562760176',
  'priority': 'u=0, i',
  'referer': 'https://www.douban.com/group/460410/',
  'sec-ch-ua': '"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"Windows"',
  'sec-fetch-dest': 'document',
  'sec-fetch-mode': 'navigate',
  'sec-fetch-site': 'same-origin',
  'sec-fetch-user': '?1',
  'upgrade-insecure-requests': '1',
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36'
}

In [12]:
#豆瓣小组爬取
group = '305169'  #豆瓣小组号
start = 1  # 从第几页开始爬取
end = 1 # 在第几页结束爬取

headlines = []
web_urls = []
dates = []
authors = []
post_content = []
comments_list = []

requested_urls = set()  #储存已请求过的链接，避免重复爬取同一个链接

# 逐页爬取标题、超链接、日期、作者名
for page in range(start, end + 1):
    url = f'https://www.douban.com/group/{group}/discussion?start={(page - 1) * 25}&type=new'
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    rows = soup.select('tr')  # 查找所有行
    for row in rows:
        title_element = row.select_one('td.title > a')   
        author_element = row.select_one('td:nth-child(2) > a')

        if title_element and time_element and author_element:
            link = title_element['href']
            
            if link not in requested_urls:
                headlines.append(title_element['title'])
                web_urls.append(title_element['href'])
                authors.append(author_element.text.strip())
                requested_urls.add(link)

            # 请求访问每个超链接（即每页上的每个帖子内容），确保只请求一次
                post_response = requests.get(title_element['href'], headers=headers)
                post_soup = BeautifulSoup(post_response.text, 'html.parser')

                text_content = ""
                image_content = post_soup.find_all('img')  #正文是否为图像

                #提取发布时间
                time_element = post_soup.select_one('.create-time')
                if time_element:
                    time_value = time_element.text.strip()
                    dates.append(time_value)   
                else:
                    dates.append("no time information")

            # 提取正文文本
                script = post_soup.find('script', type='application/ld+json')
                if script:
                    json_str = script.string.replace('\n', '').replace('\r', '').strip()
                    try:
                        json_data = json.loads(json_str)
                        text_content = json_data.get('text', '')
                    except json.JSONDecodeError as e:
                        text_content = "mistake"
                else:
                    text_content = "NA"

                if text_content !="mistake" and text_content != "NA" and text_content != "" :   #正文为正常内容
                    post_content.append(text_content)
                elif text_content == "mistake":  #正文Json解析错误
                    post_content.append("JSON解析错误")
                elif text_content == "NA":   #无正文内容
                    post_content.append("None")
                elif image_content and text_content == "":  #正文只有图片
                    post_content.append("image content")  
                else:                                       #其他可能情况
                    post_content.append("No vaild content")
            

            # 提取评论
                comments = []
                for comment_item in post_soup.find_all('li', class_='comment-item'):
                    if '不可见' in comment_item.text:
                        continue  # 跳过不可见的评论
                    reply_content = comment_item.find('div', class_='reply-content')
                    if reply_content:
                        comment_text = reply_content.get_text(strip=True)
                        comments.append(comment_text)
                if not comments:
                    comments_list.append("NA")  #无评论的情况
                else:
                # 为每条评论添加计数并连接成字符串
                    formatted_comments = " | ".join(comments) 
                    comments_list.append(formatted_comments)


# 导出为表前，检查每项列表的长度是否不一致
if not (len(headlines) == len(authors) == len(dates) == len(web_urls) == len(post_content) == len(comments_list)):
    print("数据长度不一致，请检查抓取逻辑")
    print(f"提取的标题数量: {len(headlines)}, 作者数量: {len(authors)}, 日期数量: {len(dates)}, URL数量: {len(web_urls)}, 内容数量: {len(post_content)}, 评论数量: {len(comments_list)}")

# 将上述列表变为 DataFrame
douban_data = pandas.DataFrame({
    'title': headlines,
    'username': authors,
    'post_time': dates,
    'post_content': post_content,
    'replies': comments_list,
    'url': web_urls
})

douban_data

,title,username,post_time,post_content,replies,url
0,被骗（或者不是被骗？）的献血经历,大白兔奶糖,2023-07-25 14:08:55,坐标北京，一个星期前晚上在一个区域性的闲置共享群里看到一条消息：有愿意献血的朋友吗？要O型。...,NA,https://www.douban.com/group/topic/292261610/
1,揭露微博财经大V骗术,豆友271839450,2023-07-12 14:32:05,微博财经大V骗我25万做资金盘，郑赐耿和微博财经大V史庆皓合伙诈骗，我今年回国后威胁要报警，...,NA,https://www.douban.com/group/topic/291584909/
2,昨天刚刚被骗7500，贫苦大学生欲哭无泪！,momo,2023-01-30 20:17:46,昨天下午玩游戏碰到有人前来询问购买游戏账号，之后QQ交流之后在对方给的平台上架了游戏账号进行...,报警啊，自己解决有啥用 | 现实就是 报警没用，打电话给银行客服，她只会让你对她的服务稍后做...,https://www.douban.com/group/topic/282394200/
3,被骗经历分享和互相鼓励安慰群,luckygirl127,2023-02-13 17:30:20,楼主于2月7日被电信骗了8万元，心疼和难过，相信和我一样的憎恨骗子的被骗的友友们也不少，因此...,就在这里分享一下经历啊,https://www.douban.com/group/topic/283263036/
4,很气，学生党抖音兼职被骗,momo,2020-10-01 16:13:55,image content,dd | 视角不同吧。你以为你在兼职，劳动关系；而对方则包装成经纪服务，给你做孵化引流，劳务...,https://www.douban.com/group/topic/195796372/
5,想问一下如何追回钱？报警没用12315举报也没用？谁有更好的方法？,(╯·⚇·╰)女孩,2022-12-14 15:25:13,image content,你有他身份证信息手机号这些，又有证据，起诉他是最好的，除非他的身份证是假的你就没有办法，走法...,https://www.douban.com/group/topic/280139622/
6,今天遇到一个骗子,不是不还真困难,2022-12-07 16:53:43,下次要装蠢一点，不能挑刺，不然这些骗子一点耐心都没有！难受，能不能有点好玩点的骗子,我也遇到了，但是我不知道他们怎么骗钱啊 | 估计就是给个有钱的账号你让你赚钱，然后勾起你的赚...,https://www.douban.com/group/topic/279763841/
7,养生的套路，总能很轻易的骗到老年人,江浅山,2022-09-10 01:50:42,我爸最近迷上了一个伪装成养生节目的药酒广告，就是一个所谓的“教授”当“嘉宾”跟“主持人”对话...,NA,https://www.douban.com/group/topic/274696324/
8,新型诈骗，过年守好钱包别挨骗,啊东,2022-02-02 15:02:11,image content,NA,https://www.douban.com/group/topic/259134332/
9,闲鱼放租……这是什么新骗术吗？,那哥什么🐟,2022-01-13 21:27:42,闲鱼放租，一个女的私聊我要租房，奇怪的是这个妹纸久久才回复一次，最终还是加了微信。隔了一天多...,NA,https://www.douban.com/group/topic/257858551/


In [ ]:
#导出为csv 不同文件改一下文件名
douban_data.to_csv('Douban_data729922.csv',encoding='utf-8-sig',index=False)

In [44]:
#导出为excel
import openpyxl
douban_data.to_excel('Douban_data729922.xlsx', index=False, engine='openpyxl') 

以下进行基础数据清洗

In [7]:
import pandas

In [ ]:
#多个CSV文件合并
import glob

path = ' '  # 填储存CSV文件的文件夹路径
all_files = glob.glob(path + "/*.csv")

combined_df = pandas.DataFrame()

# 遍历所有CSV文件并合并
for filename in all_files:
    df = pandas.read_csv(filename,encoding='utf-8-sig')  
    combined_df = pandas.concat([combined_df, df], ignore_index=True)  

combined_df.to_csv('combined.csv', index=False, encoding='utf-8-sig')

In [ ]:
#基础数据清洗   
df = pandas.read_csv('C:/Users/茜.NAY-LAPTOP/Desktop/combined.csv', encoding='utf-8-sig')   #填CSV文件的路径

# 删除正文和评论同时都没有内容的行
cleaned_df = df[~((df['Content'] == 'NA') & (df['Comment'] == 'NA'))]
# 删除正文是image_content 同时评论也没有内容的行 
cleaned_df = df[~((df['Content'] == 'image content') & (df['Comment'] == 'NA'))]

cleaned_df.to_csv('cleaned_file.csv', index=False, encoding='utf-8-sig')  